＃データ品質の向上

**学習目標**


1.欠落している値を解決します
2.日付機能列を日時形式に変換します
3.機能列の名前を変更し、機能列から値を削除します
4.ワンホットエンコーディング機能を作成します
5.時間的特徴の変換を理解する


＃＃ 序章

機械学習モデルは数値データのみを消費でき、数値データは「1」または「0」である必要があることを思い出してください。データに属性値がなく、ノイズや外れ値が含まれ、重複、間違ったデータ、大文字/小文字の列名があり、本質的に機械学習アルゴリズムによる取り込みの準備ができていない場合、データは「乱雑」または「乱雑」であると言われます。

このノートブックは、「乱雑な」データの最も一般的な問題のいくつかを提示して解決します。問題が異なれば必要な方法も異なり、このノートブックの範囲を超えていることに注意してください。

各学習目標は、この学生ラボノートブックの_＃TODO_に対応します。最初にこのノートブックに記入してから、[ソリューションノートブック]（https://github.com/GoogleCloudPlatform/training-data-analyst/blob/ master / tools / machine_learning / deepdive2 / launching_into_ml / Solutions / improve_data_quality.ipynb）。

In [1]:
!sudo chown -R jupyter:jupyter /home/jupyter/training-data-analyst

このラボに必要なライブラリをインポートすることから始めます

### Import Libraries

In [3]:
#必要なテンソルフローライブラリをインポートし、TFバージョンを印刷します。
import tensorflow as tf

print("TensorFlow version: ",tf.version.VERSION)


TensorFlow version:  2.1.4


In [4]:
import os
import pandas as pd  #最初に、データ処理およびCSVファイルI / OライブラリであるPandasをインポートします
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

###データセットをロードする

このデータセットは、カリフォルニアの[郵便番号別の車両燃料タイプ数]（https://data.ca.gov/dataset/vehicle-fuel-type-count-by-zip-codeSynthetic）レポートに基づいています。データセットは、データを「乱雑」にするように変更されているため、学習目的で使用できる合成表現です。


In [5]:
if not os.path.isdir("../data/transport"):
    os.makedirs("../data/transport")

In [6]:
!gsutil cp gs://cloud-training-demos/feat_eng/transport/untidy_vehicle_data.csv ../data/transport

Copying gs://cloud-training-demos/feat_eng/transport/untidy_vehicle_data.csv...
/ [1 files][ 47.2 KiB/ 47.2 KiB]                                                
Operation completed over 1 objects/47.2 KiB.                                     


In [7]:
!ls -l ../data/transport

total 48
-rw-r--r-- 1 jupyter jupyter 48343 Jul 31 14:31 untidy_vehicle_data.csv


### データセットをPandasDataFrameに読み込みます

次に、クラウドストレージバケットからコピーしたばかりのデータセットを読み込んで、PandasDataFrameを作成しましょう。また、Pandas .head（）関数を追加して、DataFrameの上位5行のデータを表示します。 Head（）とTail（）は、データセットの調査に使用される「ベストプラクティス」関数です。

In [8]:
df_transport = pd.read_csv('../data/transport/untidy_vehicle_data.csv')
df_transport.head() #最初の5行を出力します。

,Date,Zip Code,Model Year,Fuel,Make,Light_Duty,Vehicles
0,10/1/2018,90000,2006,Gasoline,OTHER/UNK,NaN,1.0
1,10/1/2018,NaN,2014,Gasoline,NaN,Yes,1.0
2,NaN,90000,NaN,Gasoline,OTHER/UNK,Yes,NaN
3,10/1/2018,90000,2017,Gasoline,OTHER/UNK,Yes,1.0
4,10/1/2018,90000,<2006,Diesel and Diesel Hybrid,OTHER/UNK,No,55.0


### DataFrame列のデータ型

DataFrameには、異種または「混合」データ型が含まれる場合があります。つまり、一部の列は数値、一部は文字列、一部は日付などです。CSVファイルには各列に含まれるデータ型に関する情報が含まれていないため、Pandasはデータを推測します。データをロードするときのタイプ。列に数値のみが含まれている場合、Pandasはその列のデータ型を整数または浮動小数点数に設定します。

次のセルを実行して、DataFrameの情報を確認します。

In [9]:
df_transport.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Date        997 non-null    object 
 1   Zip Code    997 non-null    object 
 2   Model Year  997 non-null    object 
 3   Fuel        996 non-null    object 
 4   Make        996 non-null    object 
 5   Light_Duty  996 non-null    object 
 6   Vehicles    996 non-null    float64
dtypes: float64(1), object(6)
memory usage: 54.8+ KB


.info（）関数が示すことから、6つの文字列オブジェクトと1つのfloatオブジェクトがあります。各列の最初と最後の5行を印刷してみましょう。これで、より多くの「文字列」オブジェクト値を確実に確認できます。

In [10]:
print(df_transport)

          Date Zip Code Model Year                      Fuel       Make  \
0    10/1/2018    90000       2006                  Gasoline  OTHER/UNK   
1    10/1/2018      NaN       2014                  Gasoline        NaN   
2          NaN    90000        NaN                  Gasoline  OTHER/UNK   
3    10/1/2018    90000       2017                  Gasoline  OTHER/UNK   
4    10/1/2018    90000      <2006  Diesel and Diesel Hybrid  OTHER/UNK   
..         ...      ...        ...                       ...        ...   
994   6/7/2019    90003       2012                  Gasoline     Type_R   
995   6/8/2019    90003       2012           Hybrid Gasoline  OTHER/UNK   
996   6/9/2019    90003       2012           Hybrid Gasoline     Type_Q   
997  6/10/2019    90003       2012               Natural Gas  OTHER/UNK   
998  6/11/2019    90003       2012            Plug-in Hybrid  OTHER/UNK   

    Light_Duty  Vehicles  
0          NaN       1.0  
1          Yes       1.0  
2          Yes    

###要約統計量

この時点で、数値を含む列は1つだけです（例：車両）。数値を含む特徴については、それらの値に関連するさまざまな統計的尺度に関心があることがよくあります。 .describe（）を使用して、データフレームの数値フィールドの要約統計量を確認できます。数値機能が1つしかないため、現時点では1つの要約しか表示されないことに注意してください。

In [11]:
df_transport.describe()

,Vehicles
count,996.000000
mean,72.878514
std,229.696895
min,1.000000
25%,13.000000
50%,23.000000
75%,57.250000
max,3178.000000


.groupby（）関数を使用して、もう少しデータを調べてみましょう。

In [12]:
df_transport.groupby('Fuel').first() #毎月最初のエントリを取得します。

,Date,Zip Code,Model Year,Make,Light_Duty,Vehicles
Fuel,,,,,,
Battery Electric,10/1/2018,90000,<2006,OTHER/UNK,No,4.0
Diesel and Diesel Hybrid,10/1/2018,90000,<2006,OTHER/UNK,No,55.0
Flex-Fuel,10/14/2018,90001,2007,Type_A,Yes,78.0
Gasoline,10/1/2018,90000,2006,OTHER/UNK,Yes,1.0
Hybrid Gasoline,10/24/2018,90001,2009,OTHER/UNK,Yes,18.0
Natural Gas,10/25/2018,90001,2009,OTHER/UNK,No,2.0
Other,10/8/2018,90000,<2006,OTHER/UNK,Yes,6.0
Plug-in Hybrid,11/2/2018,90001,2012,OTHER/UNK,Yes,1.0


####欠落値のチェック

値が欠落していると、機械学習モデルがデータについて不正確な推論を行う可能性があるため、データ品質に悪影響を及ぼします。欠落している値は、多くの要因の結果である可能性があります。ストリーミング送信、データ入力、またはユーザーがフィールドへの入力を忘れたときに失われた「ビット」。 Pandasは、空のセルと「NaN」タイプの両方を欠落値として認識することに注意してください。

DataFrameのすべての機能のnull値を表示してみましょう。

In [13]:
df_transport.isnull().sum()

Date          2
Zip Code      2
Model Year    2
Fuel          3
Make          3
Light_Duty    3
Vehicles      3
dtype: int64

欠落している値のサンプルを確認するには、フィーチャ列名を入力します。 「False」と「True」は、インデックス番号による値の有無に対応していることに気付くでしょう。


In [14]:
print (df_transport['Date'])
print (df_transport['Date'].isnull())

0      10/1/2018
1      10/1/2018
2            NaN
3      10/1/2018
4      10/1/2018
         ...    
994     6/7/2019
995     6/8/2019
996     6/9/2019
997    6/10/2019
998    6/11/2019
Name: Date, Length: 999, dtype: object
0      False
1      False
2       True
3      False
4      False
       ...  
994    False
995    False
996    False
997    False
998    False
Name: Date, Length: 999, dtype: bool


In [15]:
print (df_transport['Make'])
print (df_transport['Make'].isnull())

0      OTHER/UNK
1            NaN
2      OTHER/UNK
3      OTHER/UNK
4      OTHER/UNK
         ...    
994       Type_R
995    OTHER/UNK
996       Type_Q
997    OTHER/UNK
998    OTHER/UNK
Name: Make, Length: 999, dtype: object
0      False
1       True
2      False
3      False
4      False
       ...  
994    False
995    False
996    False
997    False
998    False
Name: Make, Length: 999, dtype: bool


In [16]:
print (df_transport['Model Year'])
print (df_transport['Model Year'].isnull())

0       2006
1       2014
2        NaN
3       2017
4      <2006
       ...  
994     2012
995     2012
996     2012
997     2012
998     2012
Name: Model Year, Length: 999, dtype: object
0      False
1      False
2       True
3      False
4      False
       ...  
994    False
995    False
996    False
997    False
998    False
Name: Model Year, Length: 999, dtype: bool


###この時点でデータについて何を推測できますか？

まず、行、列、機能、一意の値、欠落している値ごとにデータを要約しましょう。

In [17]:
print ("Rows     : " ,df_transport.shape[0])
print ("Columns  : " ,df_transport.shape[1])
print ("\nFeatures : \n" ,df_transport.columns.tolist())
print ("\nUnique values :  \n",df_transport.nunique())
print ("\nMissing values :  ", df_transport.isnull().sum().values.sum())


Rows     :  999
Columns  :  7

Features : 
 ['Date', 'Zip Code', 'Model Year', 'Fuel', 'Make', 'Light_Duty', 'Vehicles']

Unique values :  
 Date          248
Zip Code        6
Model Year     15
Fuel            8
Make           43
Light_Duty      3
Vehicles      210
dtype: int64

Missing values :   18


Let's see the data again -- this time the last five rows in the dataset.

In [18]:
df_transport.tail()

,Date,Zip Code,Model Year,Fuel,Make,Light_Duty,Vehicles
994,6/7/2019,90003,2012,Gasoline,Type_R,Yes,26.0
995,6/8/2019,90003,2012,Hybrid Gasoline,OTHER/UNK,Yes,4.0
996,6/9/2019,90003,2012,Hybrid Gasoline,Type_Q,Yes,25.0
997,6/10/2019,90003,2012,Natural Gas,OTHER/UNK,Yes,1.0
998,6/11/2019,90003,2012,Plug-in Hybrid,OTHER/UNK,Yes,3.0


###データ品質の問題は何ですか？

1. **データ品質の問題＃1 **：
> **欠測値**：
各機能列には、複数の欠落値があります。実際、合計18個の欠落値があります。
2。**データ品質の問題＃2 **：
> ** Date DataType **：日付は「オブジェクト」データ型として表示され、日時である必要があります。さらに、日付は1つの列にあります。私たちのビジネス要件は、日付を年、月、日に解析することです。
3。**データ品質の問題＃3 **：
> **モデルイヤー**：「<2006」ではなく、2006年を超える年にのみ関心があります。
4。**データ品質の問題＃4 **：
> **カテゴリ列**：機能列「Light_Duty」はカテゴリであり、「はい/いいえ」を選択できます。このような値を機械学習モデルにフィードすることはできません。さらに、残りの「文字列」/「オブジェクト」列を「ワンホットエンコード」する必要があります。
5. **データ品質の問題＃5 **：
> **時間的特徴**：年、月、日をどのように処理しますか？

####データ品質の問題＃1：
#####欠落している値の解決

ほとんどのアルゴリズムは欠落値を受け入れません。ただし、データセットに欠損値が見られる場合は、常に、欠損値のある「すべての行を削除」する傾向があります。パンダは空白を「NaN」で埋めますが、何らかの方法で「処理」する必要があります。

欠落値を処理するためのすべての方法はこのラボの範囲を超えていますが、考慮すべきいくつかの方法があります。数値列の場合、「平均」値を使用して、欠落している数値を埋めます。カテゴリ列の場合、「モード」（または最も頻繁な値）を使用して、欠落しているカテゴリ値を入力します。

このラボでは、.apply関数とLambda関数を使用して、すべての列に独自の最も頻繁な値を入力します。 Lambda関数については、ラボの後半で詳しく学習します。

各特徴列のNaN値が含まれている行の数を示して、欠落している値をもう一度確認しましょう。

**ラボタスク＃1a：**各機能列のNaN値を含む行数を表示して、欠落している値を確認します。

In [ ]:
# TODO 1a
# TODO -- Your code here.


**ラボタスク＃1b：**ラムダ関数を適用します。

In [ ]:
# TODO 1b
# TODO -- Your code here.


**ラボタスク＃1c：**不足している値がないかもう一度確認します。

In [ ]:
# TODO 1c
# TODO -- Your code here.


####データ品質の問題＃2：
#####日付機能列を日時形式に変換する

日付列は実際に文字列オブジェクトとして表示されます。

**ラボタスク＃2a：** Pandasのto_datetime（）関数を使用してdatetimeデータ型を変換します。

In [ ]:
# TODO 2a
# TODO -- Your code here.


**ラボタスク＃2b：**変換された日付を表示します。

In [ ]:
# TODO 2b
# TODO -- Your code here.


日付を3つの列に解析してみましょう。年、月、日。

In [ ]:
df_transport['year'] = df_transport['Date'].dt.year
df_transport['month'] = df_transport['Date'].dt.month
df_transport['day'] = df_transport['Date'].dt.day
#df ['hour'] = df ['date']。dt.hour-日付形式に時間が含まれている場合は、これを使用できます。
#df ['minute'] = df ['date']。dt.minute-日付形式に分が含まれている場合は、これを使用できます。
df_transport.info()

次に、日付の解析を確認しましょう。これにより、データの別の視覚化も可能になります。

In [ ]:
#ここでは、「grouped_data」という新しいデータフレームを作成し、「Make」列でグループ化します。
grouped_data = df_transport.groupby(['Make'])

# 毎月最初のエントリを取得します。
df_transport.groupby('month').first()

整数としての日付があるので、いくつかの追加のプロットを行いましょう。

In [ ]:
plt.figure(figsize=(10,6))
sns.jointplot(x='month',y='Vehicles',data=df_transport)
#plt.title('Vehicles by Month')

####データ品質の問題＃3：
#####フィーチャ列の名前を変更し、値を削除します。

機能列の名前には、さまざまな「大文字」が含まれています。大文字と小文字の両方の「大文字と小文字」。さらに、一部の列名には「スペース」があります。さらに、「<2006」ではなく、2006年を超える年にのみ関心があります。

**ラボタスク＃3a：**名前を変更して、フィーチャ列のすべてのスペースを削除します。

In [ ]:
# TODO 3a
# TODO -- Your code here.


**注：**次に、「SettingWithCopyWarning：値がDataFrameからのスライスのコピーに設定されようとしています」という警告を回避するために、データフレームのコピーを作成します。セルを実行して、モデルイヤー機能列から値 '<2006'を削除します。

**ラボタスク＃3b：**コピー警告の問題を回避するために、データフレームのコピーを作成します。

In [ ]:
# TODO 3b
# TODO -- Your code here.


次に、値カウントを実行して、モデルイヤー値 '<2006'が削除されたことを確認します。

In [ ]:
df['modelyear'].value_counts(0)

####データ品質の問題＃4：
#####カテゴリ列の処理

機能列「lightduty」はカテゴリ別であり、「はい/いいえ」の選択肢があります。このような値を機械学習モデルにフィードすることはできません。バイナリの回答をyes / noの文字列から1/0の整数に変換する必要があります。これを実現するにはさまざまな方法があります。ラムダ式で「apply」メソッドを使用します。パンダ。 apply（）は関数を受け取り、それをPandasシリーズのすべての値に適用します。

#####ラムダ関数とは何ですか？

通常、Pythonではdefキーワードを使用して関数を定義する必要があります。ただし、ラムダ関数は無名です。つまり、名前を付ける必要はありません。ラムダ関数の最も一般的な使用例は、単純な1行の関数を必要とするコードです（たとえば、ラムダには1つの式しかありません）。

コーススペシャライゼーションを進めると、ラムダ関数が使用されている多くの例が表示されます。今がそれらに精通する良い機会です。

まず、「軽量」機能の列にある「はい」と「いいえ」の数を数えましょう。

In [ ]:
df['lightduty'].value_counts(0)

はいを1に、いいえを0に変換しましょう。Pandas. apply() 。 applyは関数を受け取り、それをPandasシリーズのすべての値（例：lightduty）に適用します。

In [ ]:
df.loc[:,'lightduty'] = df['lightduty'].apply(lambda x: 0 if x=='No' else 1)
df['lightduty'].value_counts(0)

In [ ]:
# Confirm that "lightduty" has been converted.

df.head()

####ワンホットエンコーディングカテゴリ機能列

機械学習アルゴリズムは、カテゴリの特徴ではなく、入力ベクトルを期待します。具体的には、テキストまたは文字列値を処理できません。したがって、カテゴリの特徴をベクトルに変換すると便利なことがよくあります。

変換方法の1つは、カテゴリ機能のダミー変数を作成することです。ダミー変数は、それぞれがカテゴリ機能からの単一のクラスを表すバイナリ（0または1）変数のセットです。カテゴリ変数をワンホットベクトル、つまり1つの要素のみが非ゼロまたはホットであるベクトルとしてエンコードするだけです。ワンホットエンコーディングを使用すると、カテゴリフィーチャは配列になり、そのサイズはそのフィーチャの可能な選択肢の数になります。

Panda provides a function called "get_dummies" to convert a categorical variable into dummy/indicator variables.

In [ ]:
# Making dummy variables for categorical data with more inputs.  

data_dummy = pd.get_dummies(df[['zipcode','modelyear', 'fuel', 'make']], drop_first=True)
data_dummy.head()

**Lab Task #4a:** Merge (concatenate) original data frame with 'dummy' dataframe.

In [ ]:
# TODO 4a
# TODO -- Your code here.


**Lab Task #4b:** Drop attributes for which we made dummy variables.

In [ ]:
# TODO 4b
# TODO -- Your code here.


In [ ]:
# Confirm that 'zipcode','modelyear', 'fuel', and 'make' have been dropped.

df.head()

####データ品質の問題＃5：
#####時間的特徴列

データセットには、年、月、日の特徴列が含まれています。月と日の特徴の列を意味のある表現に変換して、時間的特徴の変更について考えさせましょう。見落とされがちです。

このスペシャライゼーションの機能エンジニアリングコースでは、年、月、日、および時間の機能列を処理する方法についてさらに詳しく説明します。

まず、データセットの「月」と「日」の一意の値を出力しましょう。

In [ ]:
print ('Unique values of month:',df.month.unique())
print ('Unique values of day:',df.day.unique())
print ('Unique values of year:',df.year.unique())

次に、各一時変数を円にマッピングして、その変数の最小値が最大値のすぐ隣に表示されるようにします。 sinおよびcos三角関数を使用して、その点のx成分とy成分を計算します。心配しないでください。TensorFlowとKerasのこれらの時間的特徴列に対処するための入力パイプラインを開発できるため、このコードを使用するのはこれが最後です。これははるかに簡単です。ただし、コースを進むときに遭遇しないことを理解する必要がある場合があります。

セルを実行して出力を表示します。

**Lab Task #5:** Drop month, and day

In [ ]:
df['day_sin'] = np.sin(df.day*(2.*np.pi/31))
df['day_cos'] = np.cos(df.day*(2.*np.pi/31))
df['month_sin'] = np.sin((df.month-1)*(2.*np.pi/12))
df['month_cos'] = np.cos((df.month-1)*(2.*np.pi/12))

# TODO 5
# TODO -- Your code here.


In [ ]:
# scroll left to see the converted month and day coluumns.
df.tail(4)

＃＃＃ 結論

このノートブックでは、データ品質を向上させるためのいくつかの概念が導入されました。欠落している値を解決し、日付機能列を日時形式に変換し、機能列の名前を変更し、機能列から値を削除し、ワンホットエンコーディング機能を作成し、時間機能を意味のある表現に変換しました。ラボの終わりまでに、機械学習モデルに入力する前にデータを「クリーンアップ」および「前処理」する必要がある理由について理解を深めました。

Copyright 2020 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.